In [86]:
"""
!pip install oracledb
!pip install gensim
!pip install cmake
!pip install kiwipiepy
!pip install pandas
!pip install tqdm
"""

'\n!pip install oracledb\n!pip install gensim\n!pip install cmake\n!pip install kiwipiepy\n!pip install pandas\n!pip install tqdm\n'

In [87]:
import os
import oracledb
import time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from kiwipiepy import Kiwi
import pickle

In [88]:
dsn1 = oracledb. makedsn("localhost",1521, service_name="XEPDB1")
username= "movie_admin"
user_pass = "pass123"

connection = oracledb.connect(user = username, password= user_pass, dsn= dsn1)

connection

<oracledb.Connection to movie_admin@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=XEPDB1)))>

In [89]:
query = "SELECT num, title, director, actor, synopsis FROM movie_tbl order by num"
movie_df=pd.read_sql(query, con = connection)

movie_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_20236\3433686458.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_df=pd.read_sql(query, con = connection)


,NUM,TITLE,DIRECTOR,ACTOR,SYNOPSIS
0,1,파묘,장재현,"최민식,김고은,유해진,이도현,김재철,김민준,김병오,전진기,박정자,박지일,이종구,이영...",미국 LA 거액의 의뢰를 받은 무당 화림(김고은)과 봉길(이도현)은 기이한 병이 대...
1,2,범죄도시4,허명행,"김무열,박지환,이동휘,마동석,곽자형,이범수,김민재,이지훈,이주빈,김도건,김지훈,현봉...",신종 마약 사건 3년 뒤 괴물형사 마석도(마동석)와 서울 광수대는 배달앱을 이용한 ...
2,3,인사이드 아웃 2,켈시 만,"에이미 포엘러,토니 헤일",디즈니픽사의 대표작 인사이드 아웃 새로운 감정과 함께 돌아오다 13살이 된 라일리의...
3,4,베테랑2,류승완,"황정민,정해인,장윤주,정만식,신승환,오달수,오대환,김시후,안보현,진경,권해효,변홍준...",가족들도 못 챙기고 밤낮없이 범죄들과 싸우는 베테랑 형사 서도철(황정민)과 강력범죄...
4,5,파일럿,김한결,"조정석,이주명,한선화,신승호,오민애,김지현,서재희,박다온,현봉식,서현철,유재석,조세...",하루 아침에 인생 추락한 스타 파일럿 제 2의 인생 이륙 준비 중 최고의 비행 실력...


In [90]:
kiwi = Kiwi()

In [91]:
kiwi.tokenize("아버지가 방에 들어가신다 그래서 슬프다 배가 아파요")

[Token(form='아버지', tag='NNG', start=0, len=3),
 Token(form='가', tag='JKS', start=3, len=1),
 Token(form='방', tag='NNG', start=5, len=1),
 Token(form='에', tag='JKB', start=6, len=1),
 Token(form='들어가', tag='VV', start=8, len=3),
 Token(form='시', tag='EP', start=11, len=1),
 Token(form='ᆫ다', tag='EF', start=11, len=2),
 Token(form='그래서', tag='MAJ', start=14, len=3),
 Token(form='슬프', tag='VA', start=18, len=2),
 Token(form='다', tag='EC', start=20, len=1),
 Token(form='배', tag='NNG', start=22, len=1),
 Token(form='가', tag='JKS', start=23, len=1),
 Token(form='아프', tag='VA', start=25, len=2),
 Token(form='어요', tag='EF', start=26, len=2)]

In [92]:
def preprocessingText(text):
    stems = []
    tagged_review = kiwi.tokenize(text)
    for token in tagged_review:
        word, pos = token.form, token.tag
        if len(word) >= 2 and (pos == "NNG" or pos == "NNP" or pos == "VA"):
            stems.append(word)
    return " ".join(stems)

In [93]:
preprocessingText("아버지가 방에 들어가신다 그래서 슬펐다 배가 아파요")

'아버지 슬프 아프'

In [94]:
movie_df.loc[1, 'SYNOPSIS']

'신종 마약 사건 3년 뒤 괴물형사 마석도(마동석)와 서울 광수대는 배달앱을 이용한 마약 판매 사건을 수사하던 중 수배 중인 앱 개발자가 필리핀에서 사망한 사건이 대규모 온라인 불법 도박 조직과 연관되어 있음을 알아낸다 필리핀에 거점을 두고 납치 감금 폭행 살인 등으로 대한민국 온라인 불법 도박 시장을 장악한 특수부대 용병 출신의 빌런 백창기(김무열)와 한국에서 더 큰 판을 짜고 있는 IT업계 천재 CEO 장동철(이동휘) 마석도는 더 커진 판을 잡기 위해 장이수(박지환)에게 뜻밖의 협력을 제안하고 광역수사대는 물론 사이버수사대까지 합류해 범죄를 소탕하기 시작하는데 나쁜 놈 잡는데 국경도 영역도 제한 없다 업그레이드 소탕 작전 거침없이 싹 쓸어버린다'

In [95]:
preprocessingText(movie_df.loc[1, 'SYNOPSIS'])

'신종 마약 사건 괴물 형사 마석 마동석 서울 광수 배달 이용 마약 판매 사건 수사 수배 개발자 필리핀 사망 사건 규모 온라인 불법 도박 조직 연관 필리핀 거점 납치 감금 폭행 살인 대한민국 온라인 불법 도박 시장 장악 특수부대 용병 출신 빌런 백창기 김무열 한국 업계 천재 동철 이동휘 마석 지환 뜻밖 협력 제안 광역 수사대 물론 사이버 수사대 합류 범죄 소탕 시작 나쁘 국경 영역 제한 업그레이드 소탕 작전'

In [96]:
movie_df["SYNOPSIS_CLEAR"]=np.NaN

In [97]:
movie_df.head()

,NUM,TITLE,DIRECTOR,ACTOR,SYNOPSIS,SYNOPSIS_CLEAR
0,1,파묘,장재현,"최민식,김고은,유해진,이도현,김재철,김민준,김병오,전진기,박정자,박지일,이종구,이영...",미국 LA 거액의 의뢰를 받은 무당 화림(김고은)과 봉길(이도현)은 기이한 병이 대...,NaN
1,2,범죄도시4,허명행,"김무열,박지환,이동휘,마동석,곽자형,이범수,김민재,이지훈,이주빈,김도건,김지훈,현봉...",신종 마약 사건 3년 뒤 괴물형사 마석도(마동석)와 서울 광수대는 배달앱을 이용한 ...,NaN
2,3,인사이드 아웃 2,켈시 만,"에이미 포엘러,토니 헤일",디즈니픽사의 대표작 인사이드 아웃 새로운 감정과 함께 돌아오다 13살이 된 라일리의...,NaN
3,4,베테랑2,류승완,"황정민,정해인,장윤주,정만식,신승환,오달수,오대환,김시후,안보현,진경,권해효,변홍준...",가족들도 못 챙기고 밤낮없이 범죄들과 싸우는 베테랑 형사 서도철(황정민)과 강력범죄...,NaN
4,5,파일럿,김한결,"조정석,이주명,한선화,신승호,오민애,김지현,서재희,박다온,현봉식,서현철,유재석,조세...",하루 아침에 인생 추락한 스타 파일럿 제 2의 인생 이륙 준비 중 최고의 비행 실력...,NaN


In [98]:
from tqdm.notebook import tqdm

sum = 0
for i in tqdm(range(10000000)):
    sum=sum +i

  0%|          | 0/10000000 [00:00<?, ?it/s]

In [99]:
len(movie_df)

485

In [100]:
row_num = len(movie_df)

In [101]:
row_num

485

In [102]:
for index in tqdm(range(row_num)):
    try:
        synopsis = movie_df.loc[index, "SYNOPSIS"]
        movie_df.loc[index, "SYNOPSISCLEAR"]= preprocessingText(synopsis)
    except Exception as e:
        movie_df.loc[index, "SYNOPSISCLEAR"]= np.NaN

  0%|          | 0/485 [00:00<?, ?it/s]

In [103]:
movie_df.head()

,NUM,TITLE,DIRECTOR,ACTOR,SYNOPSIS,SYNOPSIS_CLEAR,SYNOPSISCLEAR
0,1,파묘,장재현,"최민식,김고은,유해진,이도현,김재철,김민준,김병오,전진기,박정자,박지일,이종구,이영...",미국 LA 거액의 의뢰를 받은 무당 화림(김고은)과 봉길(이도현)은 기이한 병이 대...,NaN,미국 거액 의뢰 무당 화림 김고은 봉길 이도현 대물림 집안 장손 조상 자리 화근 냄...
1,2,범죄도시4,허명행,"김무열,박지환,이동휘,마동석,곽자형,이범수,김민재,이지훈,이주빈,김도건,김지훈,현봉...",신종 마약 사건 3년 뒤 괴물형사 마석도(마동석)와 서울 광수대는 배달앱을 이용한 ...,NaN,신종 마약 사건 괴물 형사 마석 마동석 서울 광수 배달 이용 마약 판매 사건 수사 ...
2,3,인사이드 아웃 2,켈시 만,"에이미 포엘러,토니 헤일",디즈니픽사의 대표작 인사이드 아웃 새로운 감정과 함께 돌아오다 13살이 된 라일리의...,NaN,디즈니 픽사 대표작 인사이드 아웃 감정 라일리 행복 바쁘 머릿속 감정 컨트롤 본부 ...
3,4,베테랑2,류승완,"황정민,정해인,장윤주,정만식,신승환,오달수,오대환,김시후,안보현,진경,권해효,변홍준...",가족들도 못 챙기고 밤낮없이 범죄들과 싸우는 베테랑 형사 서도철(황정민)과 강력범죄...,NaN,가족 범죄 베테랑 형사 서도철 황정민 강력 범죄 수사대 형사 교수 죽음 이전 발생 ...
4,5,파일럿,김한결,"조정석,이주명,한선화,신승호,오민애,김지현,서재희,박다온,현봉식,서현철,유재석,조세...",하루 아침에 인생 추락한 스타 파일럿 제 2의 인생 이륙 준비 중 최고의 비행 실력...,NaN,하루 아침 인생 추락 스타 파일럿 인생 이륙 준비 최고 비행 실력 스타 파일럿 인기...


In [104]:
movie_df.loc[0, "SYNOPSISCLEAR"]

'미국 거액 의뢰 무당 화림 김고은 봉길 이도현 대물림 집안 장손 조상 자리 화근 냄새 최고 풍수 상덕 최민식 장의사 유해진 합류 사람 악지 자리 상덕 기운 제안 거절 화림 설득 결국 파묘 시작'

In [105]:
movie_df["SYNOPSISCLEAR"].str.split(" ")

0      [미국, 거액, 의뢰, 무당, 화림, 김고은, 봉길, 이도현, 대물림, 집안, 장손...
1      [신종, 마약, 사건, 괴물, 형사, 마석, 마동석, 서울, 광수, 배달, 이용, ...
2      [디즈니, 픽사, 대표작, 인사이드, 아웃, 감정, 라일리, 행복, 바쁘, 머릿속,...
3      [가족, 범죄, 베테랑, 형사, 서도철, 황정민, 강력, 범죄, 수사대, 형사, 교...
4      [하루, 아침, 인생, 추락, 스타, 파일럿, 인생, 이륙, 준비, 최고, 비행, ...
                             ...                        
480    [중년, 영화감독, 오랜만, 인테리어, 디자인, 여자, 건물, 인테리어, 디자인, ...
481    [삼류, 건달, 강재, 막장, 인생, 마지막, 찬스, 평생, 보스, 대신, 감옥, ...
482    [대결, 세상, 남자, 최동원, 선동열, 고독, 치열, 맞대결, 불안, 격동, 프로...
483    [조직, 스파이, 색출, 사냥, 사냥감, 망명, 신청, 북한, 고위, 관리, 정보,...
484    [고구려, 신라, 백제, 분쟁, 원수, 백제, 의자왕, 앙심, 김춘추, 태종, 무열...
Name: SYNOPSISCLEAR, Length: 485, dtype: object

In [106]:
movie_df["SYNOPSISCLEAR_LIST"]= movie_df["SYNOPSISCLEAR"].str.split(" ")

In [107]:
movie_df.head()

,NUM,TITLE,DIRECTOR,ACTOR,SYNOPSIS,SYNOPSIS_CLEAR,SYNOPSISCLEAR,SYNOPSISCLEAR_LIST
0,1,파묘,장재현,"최민식,김고은,유해진,이도현,김재철,김민준,김병오,전진기,박정자,박지일,이종구,이영...",미국 LA 거액의 의뢰를 받은 무당 화림(김고은)과 봉길(이도현)은 기이한 병이 대...,NaN,미국 거액 의뢰 무당 화림 김고은 봉길 이도현 대물림 집안 장손 조상 자리 화근 냄...,"[미국, 거액, 의뢰, 무당, 화림, 김고은, 봉길, 이도현, 대물림, 집안, 장손..."
1,2,범죄도시4,허명행,"김무열,박지환,이동휘,마동석,곽자형,이범수,김민재,이지훈,이주빈,김도건,김지훈,현봉...",신종 마약 사건 3년 뒤 괴물형사 마석도(마동석)와 서울 광수대는 배달앱을 이용한 ...,NaN,신종 마약 사건 괴물 형사 마석 마동석 서울 광수 배달 이용 마약 판매 사건 수사 ...,"[신종, 마약, 사건, 괴물, 형사, 마석, 마동석, 서울, 광수, 배달, 이용, ..."
2,3,인사이드 아웃 2,켈시 만,"에이미 포엘러,토니 헤일",디즈니픽사의 대표작 인사이드 아웃 새로운 감정과 함께 돌아오다 13살이 된 라일리의...,NaN,디즈니 픽사 대표작 인사이드 아웃 감정 라일리 행복 바쁘 머릿속 감정 컨트롤 본부 ...,"[디즈니, 픽사, 대표작, 인사이드, 아웃, 감정, 라일리, 행복, 바쁘, 머릿속,..."
3,4,베테랑2,류승완,"황정민,정해인,장윤주,정만식,신승환,오달수,오대환,김시후,안보현,진경,권해효,변홍준...",가족들도 못 챙기고 밤낮없이 범죄들과 싸우는 베테랑 형사 서도철(황정민)과 강력범죄...,NaN,가족 범죄 베테랑 형사 서도철 황정민 강력 범죄 수사대 형사 교수 죽음 이전 발생 ...,"[가족, 범죄, 베테랑, 형사, 서도철, 황정민, 강력, 범죄, 수사대, 형사, 교..."
4,5,파일럿,김한결,"조정석,이주명,한선화,신승호,오민애,김지현,서재희,박다온,현봉식,서현철,유재석,조세...",하루 아침에 인생 추락한 스타 파일럿 제 2의 인생 이륙 준비 중 최고의 비행 실력...,NaN,하루 아침 인생 추락 스타 파일럿 인생 이륙 준비 최고 비행 실력 스타 파일럿 인기...,"[하루, 아침, 인생, 추락, 스타, 파일럿, 인생, 이륙, 준비, 최고, 비행, ..."


In [108]:
movie_df["SYNOPSISCLEAR_LIST"][1]

['신종',
 '마약',
 '사건',
 '괴물',
 '형사',
 '마석',
 '마동석',
 '서울',
 '광수',
 '배달',
 '이용',
 '마약',
 '판매',
 '사건',
 '수사',
 '수배',
 '개발자',
 '필리핀',
 '사망',
 '사건',
 '규모',
 '온라인',
 '불법',
 '도박',
 '조직',
 '연관',
 '필리핀',
 '거점',
 '납치',
 '감금',
 '폭행',
 '살인',
 '대한민국',
 '온라인',
 '불법',
 '도박',
 '시장',
 '장악',
 '특수부대',
 '용병',
 '출신',
 '빌런',
 '백창기',
 '김무열',
 '한국',
 '업계',
 '천재',
 '동철',
 '이동휘',
 '마석',
 '지환',
 '뜻밖',
 '협력',
 '제안',
 '광역',
 '수사대',
 '물론',
 '사이버',
 '수사대',
 '합류',
 '범죄',
 '소탕',
 '시작',
 '나쁘',
 '국경',
 '영역',
 '제한',
 '업그레이드',
 '소탕',
 '작전']

In [109]:
from gensim.models.word2vec import Word2Vec
word2vec = Word2Vec(
    movie_df["SYNOPSISCLEAR_LIST"],
    sg=1,
    vector_size=100,
    window=5,
    workers=10
)

In [110]:
word2vec.wv.key_to_index["시작"]

0

In [111]:
word2vec.wv.key_to_index["자신"]

1

In [112]:
word2vec.wv.key_to_index["지역"]

323

In [113]:
word2vec.wv.most_similar("인연")

[('카페', 0.9982454180717468),
 ('사상', 0.9979882836341858),
 ('이후', 0.9979478120803833),
 ('이해', 0.9979263544082642),
 ('선생', 0.9979154467582703),
 ('포기', 0.99789959192276),
 ('미미', 0.9978962540626526),
 ('감동', 0.9978901743888855),
 ('형제', 0.9978867173194885),
 ('우연', 0.9978508353233337)]

In [114]:
word2vec.wv.most_similar("사랑")

[('인생', 0.9977232217788696),
 ('공부', 0.9976749420166016),
 ('순간', 0.9974607825279236),
 ('용기', 0.9974568486213684),
 ('힘들', 0.9974219799041748),
 ('기적', 0.9974181056022644),
 ('자신', 0.9974163770675659),
 ('가을', 0.9973787665367126),
 ('도움', 0.9973683953285217),
 ('지금', 0.9973328709602356)]

In [115]:
word2vec.wv.most_similar("한국")

[('대상', 0.9981342554092407),
 ('가족', 0.9979908466339111),
 ('전쟁', 0.9979839324951172),
 ('응원', 0.9979147911071777),
 ('군사', 0.9978581666946411),
 ('당시', 0.9978374242782593),
 ('선언', 0.9978338479995728),
 ('기적', 0.9978044033050537),
 ('니시무라', 0.9977604150772095),
 ('의심', 0.9977553486824036)]

In [116]:
word2vec.wv.most_similar("니시무라")

[('결국', 0.9981055855751038),
 ('현장', 0.9980387091636658),
 ('감옥', 0.99803626537323),
 ('대상', 0.9980245232582092),
 ('눈물', 0.9979565739631653),
 ('결정', 0.9979247450828552),
 ('합류', 0.9978914856910706),
 ('인사', 0.997887372970581),
 ('빠르', 0.9978753924369812),
 ('혼란', 0.9978703856468201)]

In [117]:
cursor = connection.cursor()

In [118]:
for index, row in tqdm(movie_df.iterrows(), total=len(movie_df)):
    try:
        words = row['SYNOPSISCLEAR_LIST']
        word_vectors = []
        for word in words:
            if word in word2vec.wv:
                word_vectors.append(word2vec.wv[word])
        if len(word_vectors) > 0:
            avg_vector = np.mean(word_vectors, axis=0)
        else:
            avg_vector = np.zeros(word2vec.vector_size)
        vector_blob = pickle.dumps(avg_vector)  
        update_query = """
        UPDATE movie_tbl SET synopsis_vector = :1 WHERE num = :2
        """
        cursor.execute(update_query, [vector_blob, row['NUM']])
    except Exception as e:
        print(f"오류 발생 인덱스 (index): {e}")

  0%|          | 0/485 [00:00<?, ?it/s]

In [119]:
connection.commit()

In [120]:
cursor.close()
connection.close()